## What is scipp

Containers providing multi-dimensional array with associated dicts of coordinates, masks, and attributes

* A Mantid evolution borne out of attempt to rethink data structures
* Heavily influenced by python xarray project
* C++ core with python bindings. Python is first-class element.
* Development gathered pace in 2020

## Feature Summary

* Very flexible containers with good optimistaion potential
* Supports key features `Variances`, `Histograms`, `Masking`, `Events`, `Units`, `Bin-edges`, `Slicing`, `Sample-Environment`
* Can provide a good scientific representation of data, does not force users to work in Detector-Space
* Emphasises use of built-in generic functions
* Bundles it's own plotting library
*  `Dataset`, `DataArray` are main data containers

## Feature Exhibit

There are many demos and tutorials in the scipp online [documentation](https://scipp.github.io/)




### N-d data

We take the example of a 2D numpy array with values between 1 and 100

In [ ]:
import numpy as np
data = np.arange(1.0, 101.0).reshape(10,10)
data

In scipp we attach labels to the dimensions.
This additional information helps with numerous things as we will see below.

In [ ]:
import scipp as sc
image_data = sc.Variable(dims=['y', 'x'], values=data)

In [ ]:
from scipp.plot import plot
plot(image_data)

### Coordinates and Units

In [ ]:
# Lets give our image data the correct units
image_data.unit = sc.units.counts

x = sc.Variable(['x'], values=np.arange(10), unit=sc.units.mm)
y = sc.Variable(['y'], values=np.arange(10), unit=sc.units.mm)
image = sc.DataArray(data=image_data, coords={'x':x, 'y':y})
plot(image, aspect='equal')

In [ ]:
sc.show(image)

Coords and units, not about pretty labels, give safety to help with preventable/costly mistakes. Lets see.






In [ ]:
# Unit mismatch
reference = image.copy()
normalized = image / reference
try:
    image + normalized # Caught!
except RuntimeError:
    print("Incompatible units {} and {}".format(image.unit, normalized.unit))

In [ ]:
# Coordinate mismatch
background_corrected = reference - image
plot(background_corrected)

In [ ]:
reference.coords['x'] += 4 * sc.units.mm # Detector shifted along x
plot(reference)

In [ ]:
reference - image

### Masking

In [ ]:
image2 = image.copy()
image.masks['lhs'] = image.coords['x'] < 5.0 * sc.units.mm
plot(image)

Lets make more masks...

In [ ]:
image.masks['bad-pixel'] = image.data >= 99 * sc.units.counts
plot(image)

In [ ]:
image2.masks['bad-row'] = image.coords['y'] == 6 * sc.units.mm
plot(image2)

Masks are applied with OR. But data is not zero'd until the mask has to be lost. 

In [ ]:
image += image2
plot(image)

In [ ]:
sc.to_html(image)
sc.show(image)

### Slicing

In numpy you are required to know your dimension order

In [ ]:
data[4:,:]

In [ ]:
data[:, 4:] # Or was it the other way round?

but with scipp "crop" any dimension using the dimension label as a key.

In [ ]:
plot(image['x', 4:], aspect='equal')

You can also chain the slicing operations.

In [ ]:
plot(image['y', 1:]['x', 4:], aspect='equal')

### Dynamic type control

In [ ]:
image_data

In [ ]:
image_data.astype(sc.dtype.float32)

## Compatibility 

### Mantid
* scipp data structures are not API compatible with Mantid's 
* scipp and Mantid data structures (workspaces) **are convertible**. As one-liners in some cases:
```python
ds = sc.neutron.from_mantid(a_mantid)
```
* scipp can load and use nexus files like Mantid
```python
ds = sc.neutron.load("experiment.nxs")
```
More on this topic in the [docs](https://scipp.github.io/scipp-neutron/from-mantid-to-scipp.html)

### Numpy 

scipp objects can expose their underlying arrays in a numpy compatible form.
This makes it possible to use numpy operations directly on scipp variables.

In [ ]:
x = sc.Variable(['x'], values=np.linspace(-np.pi, np.pi, 20))
y = x.copy() # empty container
np.sin(x.values, out=y.values)
plot(y)

## Packages

Conda packages for `Linux`, `OSX`, and `Windows` on [anaconda cloud](https://anaconda.org/scipp/scipp)

### Installation

Simply
<div style="padding: 20px">
  <p style="color: #ffffff; background-color: #000000; padding: 20px">
&nbsp;conda install -c conda-forge -c scipp scipp
    </p>
</div>

Interoperability with mantid is achieved by installing the `mantid-framework` package, which is an optional dependency.
It can be installed through the same channels.

<div style="padding: 20px">
  <p style="color: #ffffff; background-color: #000000; padding: 20px">
&nbsp;conda install -c conda-forge -c scipp mantid-framework
    </p>
</div>

Full installation notes [here](https://scipp.github.io/getting-started/installation.html)



## Lots more in scipp

* IO
* label-based slicing
* events/binning
* grouping and filtering operations

## Future Plans

* Scipp 0.5 release out this week
* Across technique areas Issues and priorities are already being driven by **Instrument Data Scientists**. Includes Søren Schmidt.
* Data driven development using reduction workflows
* Priorty is to support getting *Day One* instruments ready for *Hot Commissioning*
* Aligned to above, scipp is being supplimented by `ess` and `neutron` specific modules that provide bespoke tools. 
* `scipp-widgets` library also under deveopment for building-block gui additions. See [docs](https://scipp.github.io/scipp-widgets/)
* Technical short-term [roadmap](https://scipp.github.io/about/roadmap.html) already available





## Futher Reading

1. `Heybrock, Simon et al. "Scipp: Scientific Data Handling with Labeled Multi-dimensional Arrays for C++ and Python". 1 Jan. 2020 : 169 – 181`
[PDF](https://arxiv.org/pdf/2010.00257.pdf) Simon Heybrock et al. 
1. [Source Code](https://github.com/scipp/scipp)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})